# Imports

In [ ]:
from aoc import *
import re
import os
import itertools
import math
from aocd.models import Puzzle as AOCDPuzzle


def Puzzle(day, year=2024):
    return AOCDPuzzle(year=year, day=day)

# Day 1

In [ ]:
p = Puzzle(1)
p

In [ ]:
lines = p.input_data.splitlines()
pairs = [tuple(int(i) for i in line.split("  ")) for line in lines]
lists = [sorted([tpl[i] for tpl in pairs]) for i in range(2)]
p.answer_a = sum(abs(a - b) for a, b in zip(*lists))

In [ ]:
from collections import Counter

counts = Counter(lists[1])
p.answer_b = sum(val * counts[val] for val in lists[0])

# Day 2

In [ ]:
p = Puzzle(year=2024, day=2)
p

In [ ]:
reports = [vector(line) for line in p.input_data.splitlines()]


def is_safe(report):
    diffs = [(b - a) for a, b in itertools.pairwise(report)]
    return all(1 <= abs(d) <= 3 for d in diffs) and (
        all(d > 0 for d in diffs) or all(d < 0 for d in diffs)
    )


def signum(i):
    return -1 if i < 0 else 1 if i > 0 else 0


def dampen(report):
    return is_safe(report) or (
        any(is_safe(report[:i] + report[i + 1 :]) for i in range(len(report)))
    )


p.answer_a = len([r for r in reports if is_safe(r)])
p.answer_b = len([r for r in reports if dampen(r)])